In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

# from albk.data.utils import idx_to_locate
use_disjoint_files = False


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
# from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
# from astra.torch.utils import train_fn

import torchvision.models as models
# from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [6]:
def get_common_label_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    common_files = set(f1_base_files).intersection(f2_base_files)
    common_label_files = []
    for file in common_files:
        ds1 = xr.open_dataset(join(path1, file))
        ds2 = xr.open_dataset(join(path2, file))
        if np.all(ds1.label.values == ds2.label.values):
            common_label_files.append(file)
    
    return list(map(lambda f: join(path1, f), common_label_files))

def get_disjoint_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    disjoint_files = set(f1_base_files).symmetric_difference(f2_base_files)
    
    f1_disjoint = [f for f in disjoint_files if f in f1_base_files]
    f1_disjoint = list(map(lambda f: join(path1, f), f1_disjoint))

    f2_disjoint = [f for f in disjoint_files if f in f2_base_files]
    f2_disjoint = list(map(lambda f: join(path2, f), f2_disjoint))
    
    return f1_disjoint + f2_disjoint

In [8]:
base_path = expanduser("/home/rishabh.mondal/bangladesh_labels/bkdb/india_labels/region/delhi/sarath_data")
paths = {"rishabh": ("shataxi", "suraj"), "suraj": ("rishabh", "vannsh")}

all_labeled_files = []
for moderator, annotators in paths.items():
    # Get moderator files
    moderator_path = join(base_path, "moderated", moderator)
    moderator_files = glob(join(moderator_path, "*.nc"))
    
    # Get annotator common label files
    annotator1_path = join(base_path, annotators[0])
    annotator2_path = join(base_path, annotators[1])
    
    common_base_files = get_common_label_files(annotator1_path, annotator2_path)
    
    # Get disjoint files
    disjoint_files = get_disjoint_files(annotator1_path, annotator2_path)
    
    all_files = moderator_files + common_base_files
    if use_disjoint_files:
        all_files.extend(disjoint_files)
    assert len(all_files) == len(set(all_files))
    all_labeled_files.extend(all_files)
    
    print("Moderator", moderator)
    print(" "*5, "Moderator files", len(moderator_files))
    print(" "*5, "Common label files", len(common_base_files))
    print(" "*5, "Disjoint files", len(disjoint_files))
    print(" "*5, f"Total files from {moderator} and {annotators}", len(all_files))
    print(" "*5, "Total annotatated files", len(all_labeled_files))
    
print("Total dataset size", len(all_labeled_files) * 25)


Moderator rishabh
      Moderator files 151
      Common label files 98
      Disjoint files 0
      Total files from rishabh and ('shataxi', 'suraj') 249
      Total annotatated files 249
Moderator suraj
      Moderator files 88
      Common label files 64
      Disjoint files 0
      Total files from suraj and ('rishabh', 'vannsh') 152
      Total annotatated files 401
Total dataset size 10025


In [9]:
len(all_labeled_files)

401

In [10]:
all_labeled_files[0]

'/home/rishabh.mondal/bangladesh_labels/bkdb/india_labels/region/delhi/sarath_data/moderated/rishabh/28.90,77.25.nc'

In [ ]:
# create a torch dataset from the files

def process_file(file):
    raw_file_path = f"/home/rishabh.mondal/bkdb/statewise/sarath_data1/{file.split('/')[-1].rsplit('.', 1)[0]}.zarr"
    try:
        ds = xr.open_zarr(raw_file_path, consolidated=False)
    except:
        print(f"Failed to open {raw_file_path}")
        return
    label_ds = xr.open_dataset(file)
    image_label_pairs = []
    for lat_lag in [-2, -1, 0, 1, 2]:
        for lon_lag in [-2, -1, 0, 1, 2]:
            img = Image.fromarray(ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['data'].values)
            label = label_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['label'].values.item()
            # img.save(f"/tmp/bk_new_delhi/{file.split('/')[-1].rsplit('.', 1)[0]}_{lat_lag}_{lon_lag}_{label}.png")

_ = Parallel(n_jobs=32)(delayed(process_file)(file) for file in tqdm(all_labeled_files))